# Đánh Giá Mô Hình Embedding Cho Luật Tiếng Việt

Notebook này thực hiện đánh giá sơ bộ nhiều mô hình embedding cho tiếng Việt và luật tiếng Việt

## Yêu cầu:
- Đánh giá các mô hình embedding từ 512 token trở lên
- Lưu embeddings trên RAM
- Làm giàu data: 200-300 văn bản luật
- Sử dụng queries từ benchmark
- Đánh giá top 10-15 kết quả và ngưỡng điểm



In [63]:
import warnings
warnings.filterwarnings('ignore')

import torch
import numpy as np
import pandas as pd
import os
import sys
import re
import json
import time
import gc
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity

print("🔄 Importing core libraries...")

# Check installed versions
def check_version(package_name, expected_version=None):
    try:
        import importlib.metadata
        version = importlib.metadata.version(package_name)
        if expected_version and version != expected_version:
            print(f"⚠️  {package_name}: expected {expected_version}, got {version}")
        else:
            print(f"✅ {package_name}: {version}")
        return True, version
    except Exception as e:
        print(f"❌ {package_name}: not found or error ({e})")
        return False, None

# Check critical versions
check_version("huggingface_hub", "0.16.4")
check_version("tokenizers", "0.13.3")
check_version("transformers", "4.32.1")
check_version("sentence-transformers", "2.2.2")

# Import transformers with detailed error handling
print("\n🔄 Importing transformers...")
try:
    from transformers import AutoTokenizer, AutoModel
    print("✅ transformers imported successfully")
    
    # Quick functionality test
    test_tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
    test_tokens = test_tokenizer("test", return_tensors="pt")
    print("✅ transformers functionality test passed")
    del test_tokenizer, test_tokens
    
except Exception as e:
    print(f"❌ transformers failed: {e}")
    print("🔧 If this fails, please restart kernel and run first cell again")
    raise

# Import sentence-transformers with detailed error handling
print("\n🔄 Importing sentence-transformers...")
try:
    from sentence_transformers import SentenceTransformer
    print("✅ sentence-transformers imported successfully")
    
    # Quick functionality test
    test_model = SentenceTransformer("all-MiniLM-L6-v2")
    test_embedding = test_model.encode(["test sentence"])
    print(f"✅ sentence-transformers functionality test passed (embedding shape: {test_embedding.shape})")
    del test_model, test_embedding
    
except Exception as e:
    print(f"❌ sentence-transformers failed: {e}")
    print("🔧 If this fails, please restart kernel and run first cell again")
    raise

# Import document processing
print("\n🔄 Importing document processing...")
try:
    from docx import Document
    print("✅ python-docx imported successfully")
except ImportError:
    print("📦 Installing python-docx...")
    import subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "python-docx==0.8.11"])
    from docx import Document
    print("✅ python-docx installed and imported")

# Device configuration
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"\n🖥️  Using device: {device}")
if device == "cuda":
    print(f"   GPU: {torch.cuda.get_device_name()}")
    print(f"   GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

print("\n🎉 All libraries loaded successfully! Ready for evaluation.")


🔄 Importing core libraries...
⚠️  huggingface_hub: expected 0.16.4, got 0.34.4
⚠️  tokenizers: expected 0.13.3, got 0.22.0
⚠️  transformers: expected 4.32.1, got 4.56.1
⚠️  sentence-transformers: expected 2.2.2, got 5.1.0

🔄 Importing transformers...
✅ transformers imported successfully
✅ transformers functionality test passed

🔄 Importing sentence-transformers...
✅ sentence-transformers imported successfully
✅ sentence-transformers functionality test passed (embedding shape: (1, 384))

🔄 Importing document processing...
✅ python-docx imported successfully

🖥️  Using device: cuda
   GPU: NVIDIA GeForce RTX 3050 Ti Laptop GPU
   GPU Memory: 4.3 GB

🎉 All libraries loaded successfully! Ready for evaluation.


## 1. Chuẩn bị dữ liệu luật


In [64]:
# ===== ROMAN NUMBER CONVERTER =====
print("🔢 Setting up Roman numeral converter...")

ROMAN_MAP = {'I':1,'V':5,'X':10,'L':50,'C':100,'D':500,'M':1000}

def roman_to_int(s: str):
    """
    Chuyển đổi số La Mã sang số nguyên
    Ví dụ: 'I' -> 1, 'IV' -> 4, 'IX' -> 9, 'X' -> 10
    """
    s = s.upper().strip()
    if not s or any(ch not in ROMAN_MAP for ch in s):
        return None
    total = 0
    prev = 0
    for ch in reversed(s):
        val = ROMAN_MAP[ch]
        if val < prev:
            total -= val
        else:
            total += val
            prev = val
    return total

# Test Roman converter
test_romans = ['I',"II","III","IV","V","VI","VII","VIII","IX","X", "L", "C", "D", "M"]
print("✅ Roman converter test:")
for roman in test_romans:
    print(f"   {roman} -> {roman_to_int(roman)}")

print("✅ Roman numeral converter ready!")


🔢 Setting up Roman numeral converter...
✅ Roman converter test:
   I -> 1
   II -> 2
   III -> 3
   IV -> 4
   V -> 5
   VI -> 6
   VII -> 7
   VIII -> 8
   IX -> 9
   X -> 10
   L -> 50
   C -> 100
   D -> 500
   M -> 1000
✅ Roman numeral converter ready!


In [65]:
# ===== UTILITY FUNCTIONS FOR EMBEDDING =====
print("🔧 Setting up embedding utility functions...")

def mean_pooling(model_output, attention_mask):
    """
    Mean pooling để tạo sentence embeddings từ token embeddings
    
    Input:
    - model_output: output từ transformer model
    - attention_mask: mask để ignore padding tokens
    
    Output:
    - Sentence embeddings với mean pooling
    """
    token_embeddings = model_output[0]  # First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

def encode_with_transformers(texts, model_name, max_length=512, batch_size=32):
    """
    Encode texts using transformers library với mean pooling
    
    Input:
    - texts: list of texts to encode
    - model_name: HuggingFace model name
    - max_length: maximum sequence length
    - batch_size: batch size for processing
    
    Output:
    - numpy array of embeddings (stored in RAM)
    """
    print(f"Loading model: {model_name}")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name).to(device)
    model.eval()
    
    all_embeddings = []
    
    with torch.no_grad():
        for i in tqdm(range(0, len(texts), batch_size), desc="Encoding"):
            batch = texts[i:i+batch_size]
            
            # Tokenize
            encoded = tokenizer(
                batch, 
                padding=True, 
                truncation=True, 
                max_length=max_length,
                return_tensors='pt'
            ).to(device)
            
            # Get model output
            model_output = model(**encoded)
            
            # Mean pooling
            sentence_embeddings = mean_pooling(model_output, encoded['attention_mask'])
            
            # Normalize embeddings
            sentence_embeddings = torch.nn.functional.normalize(sentence_embeddings, p=2, dim=1)
            
            # Move to CPU and convert to numpy (store in RAM)
            embeddings = sentence_embeddings.cpu().numpy()
            all_embeddings.append(embeddings)
            
            # Clear GPU memory
            del encoded, model_output, sentence_embeddings, embeddings
            torch.cuda.empty_cache() if device == "cuda" else None
    
    # Delete model to free memory
    del model, tokenizer
    torch.cuda.empty_cache() if device == "cuda" else None
    
    # Combine all embeddings and store in RAM
    final_embeddings = np.vstack(all_embeddings)
    print(f"   ✅ Generated {final_embeddings.shape[0]} embeddings, stored in RAM")
    
    return final_embeddings

def encode_with_sentence_transformers(texts, model_name, batch_size=32):
    """
    Encode texts using sentence-transformers library
    
    Input:
    - texts: list of texts to encode  
    - model_name: SentenceTransformer model name
    - batch_size: batch size for processing
    
    Output:
    - numpy array of embeddings (stored in RAM)
    """
    print(f"Loading model: {model_name}")
    model = SentenceTransformer(model_name)
    model.to(device)
    
    # Encode all texts
    embeddings = model.encode(
        texts,
        batch_size=batch_size,
        show_progress_bar=True,
        convert_to_numpy=True,  # Convert to numpy (RAM storage)
        normalize_embeddings=True  # Normalize for cosine similarity
    )
    
    # Delete model to free memory
    del model
    torch.cuda.empty_cache() if device == "cuda" else None
    
    print(f"   ✅ Generated {embeddings.shape[0]} embeddings, stored in RAM")
    
    return embeddings

print("✅ Embedding utility functions ready!")
print("   📦 encode_with_transformers: for transformers models with mean pooling")
print("   📦 encode_with_sentence_transformers: for sentence-transformers models") 
print("   💾 All embeddings stored in RAM (not vector database)")


🔧 Setting up embedding utility functions...
✅ Embedding utility functions ready!
   📦 encode_with_transformers: for transformers models with mean pooling
   📦 encode_with_sentence_transformers: for sentence-transformers models
   💾 All embeddings stored in RAM (not vector database)


In [66]:
# ===== DOCUMENT READER FUNCTIONS =====
print("📖 Setting up document reading functions...")

def read_docx(file_path):
    """
    Đọc file docx và trả về text
    Input: đường dẫn file .docx
    Output: text content của file
    """
    print(f"   Reading file: {file_path}")
    doc = Document(file_path)
    text = "\n".join((p.text or "").strip() for p in doc.paragraphs)
    print(f"   ✅ Successfully read {len(text):,} characters")
    return text

def normalize_lines(text: str):
    """
    Chuẩn hóa các dòng text - loại bỏ whitespace thừa
    Input: raw text
    Output: list các dòng đã được normalize
    """
    lines = [re.sub(r'\s+$', '', ln) for ln in text.splitlines()]
    print(f"   ✅ Normalized {len(lines):,} lines")
    return lines

print("✅ Document reading functions ready!")


📖 Setting up document reading functions...
✅ Document reading functions ready!


In [67]:
# ===== PROPER ADVANCED LAW DOCUMENT CHUNKING =====
print("⚖️ Setting up PROPER advanced law document chunking (from chunking.py)...")

def advanced_chunk_law_document(text, max_length=600):
    """
    Chia văn bản luật thành chunks theo logic CHÍNH XÁC 
    
    2-Pass System:
    - Pass 1 (prescan): Đếm CHƯƠNG & ĐIỀU ở đầu dòng
    - Pass 2 (strict): Strict sequence validation + Clause intro injection
    
    KEY FEATURES:
    - Khoản: PHẢI dạng "1." (số + dấu chấm)
    - Điểm: PHẢI dạng "a)" (có dấu )). Chỉ bắt chuỗi điểm nếu mở đầu là a).
    - TIÊM intro khoản vào content của mọi điểm (QUAN TRỌNG!)
    """
    print("   🔍 2-Pass advanced parsing with full validation...")
    lines = normalize_lines(text)
    
    # ===== Regex patterns (CHÍNH XÁC từ chunking.py) =====
    ARTICLE_RE = re.compile(r'^Điều\s+(\d+)\s*[\.:]?\s*(.*)$', re.UNICODE)
    CHAPTER_RE = re.compile(r'^Chương\s+([IVXLCDM]+)\s*(.*)$', re.UNICODE|re.IGNORECASE)
    SECTION_RE = re.compile(r'^Mục\s+(\d+)\s*[:\-]?\s*(.*)$', re.UNICODE|re.IGNORECASE)
    CLAUSE_RE = re.compile(r'^\s*(\d+)\.\s*(.*)$', re.UNICODE)  # Khoản: PHẢI "1." (số + dấu chấm)
    POINT_RE = re.compile(r'^\s*([a-zA-ZđĐ])\)\s+(.*)$', re.UNICODE)  # Điểm: PHẢI "a)" (bắt buộc có ')')
    
    # ===== PASS 1: Pre-scan CHƯƠNG/ĐIỀU =====
    def prescan(lines):
        chapters_nums, articles_nums = [], []
        chapters_labels = []
        for line in lines:
            if not line: continue
            m_ch = CHAPTER_RE.match(line)
            if m_ch:
                n = roman_to_int(m_ch.group(1))
                if n:
                    chapters_nums.append(n)
                    title = (m_ch.group(2) or "").strip()
                    chapters_labels.append(f"Chương {m_ch.group(1).strip()}" + (f" – {title}" if title else ""))
                continue
            m_art = ARTICLE_RE.match(line)
            if m_art:
                num = int(m_art.group(1))
                articles_nums.append(num)
                continue
        return chapters_nums, articles_nums, chapters_labels
    
    # ===== Flush helpers =====
    def flush_article_intro(chunks, article_no, article_title, article_intro_buf, chapter, section, stats):
        content = (article_intro_buf or "").strip()
        if not content: return
        title_line = f"Điều {article_no}. {article_title}".strip() if article_title else f"Điều {article_no}"
        chunks.append({
            "content": f"{title_line}\n{content}",
            "title": f"Điều {article_no} - {article_title}" if article_title else f"Điều {article_no}",
            "type": "article_intro",
            "metadata": {
                "chapter": chapter,
                "section": section,
                "article_no": article_no,
                "article_title": article_title,
                "exact_citation": f"Điều {article_no}"
            }
        })
        stats["article_intro"] += 1
    
    def flush_clause(chunks, article_no, article_title, clause_no, content, chapter, section, stats):
        content = (content or "").strip()
        if not content: return
        chunks.append({
            "content": f"Khoản {clause_no}. {content}",
            "title": f"Điều {article_no}, Khoản {clause_no}",
            "type": "clause",
            "metadata": {
                "chapter": chapter,
                "section": section,
                "article_no": article_no,
                "article_title": article_title,
                "clause_no": clause_no,
                "exact_citation": f"Điều {article_no} khoản {clause_no}"
            }
        })
        stats["clauses"] += 1
    
    def flush_point(chunks, article_no, article_title, clause_no, letter, content, chapter, section, stats, clause_intro=None):
        content = (content or "").strip()
        if not content: return
        
        # 🔥 TIÊM intro khoản vào đầu nội dung điểm (KEY FEATURE từ chunking.py!)
        if clause_intro:
            intro = clause_intro.rstrip().rstrip(':') + ':'
            content = f"{intro}\n{content}"
        
        letter = letter.lower()
        chunks.append({
            "content": f"Khoản {clause_no}, điểm {letter}) {content}",
            "title": f"Điều {article_no}, Khoản {clause_no}, Điểm {letter})",
            "type": "point",
            "metadata": {
                "chapter": chapter,
                "section": section,
                "article_no": article_no,
                "article_title": article_title,
                "clause_no": clause_no,
                "point_letter": letter,
                "exact_citation": f"Điều {article_no} khoản {clause_no} điểm {letter})",
                "clause_intro": clause_intro  # Lưu để trace
            }
        })
        stats["points"] += 1
    
    print(f"   📄 Processing {len(lines):,} lines...")
    
    # PASS 1: Pre-scan
    print("   🔍 Pass 1: Pre-scanning structure...")
    chapters_nums, articles_nums, chapters_labels = prescan(lines)
    chapters_set, articles_set = set(chapters_nums), set(articles_nums)
    
    print(f"      - Pre-scan found {len(chapters_nums)} chapters: {chapters_nums[:10]}{'...' if len(chapters_nums)>10 else ''}")
    print(f"      - Pre-scan found {len(articles_nums)} articles: {articles_nums[:20]}{'...' if len(articles_nums)>20 else ''}")
    
    # PASS 2: Strict chunking
    print("   🔍 Pass 2: Strict parsing with sequence validation...")
    chunks = []
    stats = {"articles": 0, "article_intro": 0, "clauses": 0, "points": 0}
    warnings = []
    
    chapter_label = None
    section_label = None
    article_no = None
    article_title = ""
    expecting_article_title = False
    article_intro_buf = ""
    article_has_any_chunk = False
    
    clause_no = None
    clause_buf = ""
    clause_intro_current = None  # 🔥 Intro của khoản sẽ được tiêm vào mọi điểm
    in_points = False
    point_letter = None
    point_buf = ""
    
    expected_chapter = None
    expected_article = None
    seeking_article = False
    
    def close_clause():
        nonlocal clause_no, clause_buf, in_points, point_letter, point_buf, article_has_any_chunk, clause_intro_current
        if clause_no is None: return
        if in_points and point_letter:
            flush_point(chunks, article_no, article_title, clause_no, point_letter,
                        point_buf, chapter_label, section_label, stats, clause_intro_current)
        elif clause_buf.strip():
            flush_clause(chunks, article_no, article_title, clause_no, clause_buf,
                         chapter_label, section_label, stats)
        article_has_any_chunk = True
        clause_no, clause_buf, in_points, point_letter, point_buf = None, "", False, None, ""
        clause_intro_current = None
    
    def close_article_if_needed():
        nonlocal article_intro_buf, article_has_any_chunk
        if (not article_has_any_chunk) and article_intro_buf.strip():
            flush_article_intro(chunks, article_no, article_title, article_intro_buf,
                                chapter_label, section_label, stats)
        article_intro_buf = ""
        article_has_any_chunk = False
    
    for ln_idx, line in enumerate(lines, start=1):
        if not line: continue
        
        # Seeking article logic (strict sequence validation)
        if seeking_article:
            m_art_seek = ARTICLE_RE.match(line)
            if m_art_seek:
                a_no = int(m_art_seek.group(1))
                if a_no == expected_article:
                    seeking_article = False
                    close_clause()
                    if article_no is not None:
                        close_article_if_needed()
                    article_no = a_no
                    article_title = (m_art_seek.group(2) or "").strip()
                    stats["articles"] += 1
                    if not article_title:
                        expecting_article_title = True
                    expected_article = a_no + 1
                    clause_no = None; clause_buf = ""; in_points = False; point_letter = None; point_buf = ""
                    clause_intro_current = None
                    continue
                else:
                    continue
            m_ch_seek = CHAPTER_RE.match(line)
            if m_ch_seek:
                break  # Stop if hit new chapter while seeking
            continue
        
        # Expecting article title on next line
        if expecting_article_title:
            if not (CHAPTER_RE.match(line) or SECTION_RE.match(line) or CLAUSE_RE.match(line) or POINT_RE.match(line) or ARTICLE_RE.match(line)):
                article_title = line; expecting_article_title = False; continue
            else:
                expecting_article_title = False
        
        # CHƯƠNG (with strict sequence validation)
        m_ch = CHAPTER_RE.match(line)
        if m_ch:
            close_clause()
            if article_no is not None:
                close_article_if_needed()
            article_no = None
            article_title = ""
            article_intro_buf = ""
            expecting_article_title = False
            
            roman = m_ch.group(1).strip()
            ch_num = roman_to_int(roman) or 0
            ch_title = (m_ch.group(2) or "").strip()
            lbl = f"Chương {roman}" + (f" – {ch_title}" if ch_title else "")
            
            if expected_chapter is None:
                expected_chapter = ch_num + 1
            else:
                if ch_num == expected_chapter:
                    expected_chapter = ch_num + 1
                elif ch_num > expected_chapter:
                    if expected_chapter not in chapters_set:
                        warnings.append(f"Missing Chapter {expected_chapter} - stopping at {lbl}")
                        break
                    else:
                        warnings.append(f"Skipping {lbl} - waiting for Chapter {expected_chapter}")
                        continue
                else:
                    warnings.append(f"Skipping {lbl} - backward chapter number")
                    continue
            
            chapter_label = lbl
            section_label = None
            continue
        
        # MỤC
        m_sec = SECTION_RE.match(line)
        if m_sec:
            close_clause()
            if article_no is not None:
                close_article_if_needed()
            article_no = None
            article_title = ""
            article_intro_buf = ""
            expecting_article_title = False
            
            sec_no = m_sec.group(1).strip()
            sec_title = (m_sec.group(2) or "").strip()
            section_label = f"Mục {sec_no}" + (f" – {sec_title}" if sec_title else "")
            continue
        
        # ĐIỀU (with strict sequence validation)
        m_art = ARTICLE_RE.match(line)
        if m_art:
            a_no = int(m_art.group(1))
            a_title = (m_art.group(2) or "").strip()
            
            if expected_article is None:
                expected_article = a_no + 1
                close_clause()
                if article_no is not None: close_article_if_needed()
                article_no = a_no; article_title = a_title; stats["articles"] += 1
                if not article_title: expecting_article_title = True
                clause_no = None; clause_buf = ""; in_points = False; point_letter = None; point_buf = ""
                clause_intro_current = None
                continue
            else:
                if a_no == expected_article:
                    expected_article = a_no + 1
                    close_clause()
                    if article_no is not None: close_article_if_needed()
                    article_no = a_no; article_title = a_title; stats["articles"] += 1
                    if not article_title: expecting_article_title = True
                    clause_no = None; clause_buf = ""; in_points = False; point_letter = None; point_buf = ""
                    clause_intro_current = None
                    continue
                elif a_no > expected_article:
                    if expected_article not in articles_set:
                        warnings.append(f"Missing Article {expected_article} - stopping at Article {a_no}")
                        break
                    else:
                        warnings.append(f"Skipping Article {a_no} - waiting for Article {expected_article}")
                        seeking_article = True
                        continue
                else:
                    warnings.append(f"Skipping Article {a_no} - backward article number")
                    continue
        
        # Skip if not in any article
        if article_no is None:
            continue
        
        # KHOẢN — PHẢI "1." (số + dấu chấm) - STRICT!
        m_k = CLAUSE_RE.match(line)
        if m_k and m_k.group(1).isdigit():
            if article_intro_buf.strip():
                flush_article_intro(chunks, article_no, article_title, article_intro_buf,
                                    chapter_label, section_label, stats)
                article_intro_buf = ""; article_has_any_chunk = True
            close_clause()
            clause_no = int(m_k.group(1))
            clause_buf = (m_k.group(2) or "").strip()
            in_points = False; point_letter = None; point_buf = ""
            clause_intro_current = None
            continue
        
        # ĐIỂM — 🔥 chỉ bắt đầu chuỗi điểm nếu mở đầu là a) (KEY LOGIC!)
        m_p = POINT_RE.match(line)
        if m_p and clause_no is not None:
            letter = m_p.group(1).lower()
            text = (m_p.group(2) or "").strip()
            
            if not in_points:
                if letter != 'a':
                    # 🔥 không coi là điểm -> gộp vào nội dung khoản (KEY LOGIC!)
                    clause_buf += ("\\n" if clause_buf else "") + f"{letter}) {text}"
                    continue
                # 🔥 bắt đầu chuỗi điểm với 'a)' — KHÔNG flush chunk "Khoản X"
                # lưu intro khoản để tiêm vào MỌI điểm (KEY FEATURE!)
                clause_intro_current = clause_buf.strip() if clause_buf.strip() else None
                clause_buf = ""
                in_points = True
                point_letter = letter
                point_buf = text
                continue
            
            # đang trong chuỗi điểm: flush điểm trước, mở điểm mới
            if point_letter:
                flush_point(chunks, article_no, article_title, clause_no, point_letter,
                            point_buf, chapter_label, section_label, stats, clause_intro_current)
            in_points = True
            point_letter = letter
            point_buf = text
            continue
        
        # Nội dung kéo dài
        if clause_no is not None:
            if in_points and point_letter:
                point_buf += ("\\n" if point_buf else "") + line
            else:
                clause_buf += ("\\n" if clause_buf else "") + line
        else:
            # intro điều (chỉ tồn tại trước khi có khoản đầu tiên)
            article_intro_buf += ("\\n" if article_intro_buf else "") + line
    
    # Kết thúc file
    close_clause()
    if article_no is not None:
        close_article_if_needed()
    
    # Filter valid chunks
    final_chunks = []
    for chunk in chunks:
        content = chunk['content'].strip()
        if len(content) > 50:  # Chỉ lấy chunks đủ dài
            final_chunks.append(chunk)
    
    print(f"   📊 PROPER advanced parsing results:")
    print(f"      - Processed {stats['articles']} articles (strict sequence)")
    print(f"      - Created {stats['article_intro']} article intros") 
    print(f"      - Created {stats['clauses']} clauses")
    print(f"      - Created {stats['points']} points (with clause intro injection)")
    print(f"      - Final valid chunks: {len(final_chunks)}")
    
    if warnings:
        print(f"   ⚠️  Warnings: {len(warnings)} issues detected")
        for w in warnings[:3]:  # Show first 3 warnings
            print(f"      - {w}")
    
    return final_chunks

print("✅ PROPER Advanced law document chunking ready!")


⚖️ Setting up PROPER advanced law document chunking (from chunking.py)...
✅ PROPER Advanced law document chunking ready!


In [68]:
# ===== LOAD LAW DOCUMENT DATA =====
print("📚 Loading law document from LuatHonNhan folder...")

law_file_path = "LuatHonNhan/luat_hon_nhan_va_gia_dinh.docx"
if os.path.exists(law_file_path):
    print(f"✅ Found law file: {law_file_path}")
    
    # Bước 1: Đọc file docx
    print("\n📖 Step 1: Reading DOCX file...")
    law_text = read_docx(law_file_path)
    
    # Bước 2: Chia thành chunks với thuật toán PROPER ADVANCED 
    print("\n🔨 Step 2: PROPER Advanced chunking with 2-pass validation...")
    law_chunks = advanced_chunk_law_document(law_text, max_length=600)
    
    # Bước 3: Chuẩn bị dữ liệu cho đánh giá
    print("\n🗂️ Step 3: Preparing data for evaluation...")
    law_docs = []
    for i, chunk in enumerate(law_chunks):
        law_docs.append({
            'id': i,
            'title': chunk['title'],
            'text': chunk['content'],
            'length': len(chunk['content']),
            'type': chunk['type'],
            'metadata': chunk.get('metadata', {})
        })
    
    print(f"   ✅ Processed {len(law_docs)} law document chunks")
    print(f"   📊 Average chunk length: {np.mean([doc['length'] for doc in law_docs]):.0f} characters")
    
    # Bước 4: Thống kê theo loại
    print(f"\n📈 Step 4: Chunk distribution analysis...")
    type_counts = {}
    for doc in law_docs:
        doc_type = doc['type']
        type_counts[doc_type] = type_counts.get(doc_type, 0) + 1
    
    print(f"   📊 Chunk distribution by type:")
    for chunk_type, count in type_counts.items():
        print(f"      - {chunk_type}: {count} chunks")
    
    # Bước 5: Hiển thị examples
    print(f"\n🔍 Step 5: Sample chunks preview...")
    for i in range(min(3, len(law_docs))):
        doc = law_docs[i]
        print(f"\n   📄 Chunk {i+1}: {doc['title']}")
        print(f"      Type: {doc['type']} | Length: {doc['length']} chars")
        print(f"      Content preview: {doc['text'][:150]}...")
        if doc.get('metadata'):
            print(f"      Metadata: {doc['metadata']}")
    
    print(f"\n✅ Successfully loaded {len(law_docs)} law document chunks!")
    print(f"🎯 Dataset ready for embedding evaluation (Target: 200-300 chunks)")
        
else:
    print(f"❌ File {law_file_path} not found!")
    print("   Please make sure the LuatHonNhan folder is in the correct location")
    print(f"   Expected path: {os.path.abspath(law_file_path)}")
    law_docs = []


📚 Loading law document from LuatHonNhan folder...
✅ Found law file: LuatHonNhan/luat_hon_nhan_va_gia_dinh.docx

📖 Step 1: Reading DOCX file...
   Reading file: LuatHonNhan/luat_hon_nhan_va_gia_dinh.docx
   ✅ Successfully read 86,564 characters

🔨 Step 2: PROPER Advanced chunking with 2-pass validation...
   🔍 2-Pass advanced parsing with full validation...
   ✅ Normalized 608 lines
   📄 Processing 608 lines...
   🔍 Pass 1: Pre-scanning structure...
      - Pre-scan found 9 chapters: [1, 2, 3, 4, 5, 6, 7, 8, 9]
      - Pre-scan found 133 articles: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]...
   🔍 Pass 2: Strict parsing with sequence validation...
   📊 PROPER advanced parsing results:
      - Processed 133 articles (strict sequence)
      - Created 44 article intros
      - Created 274 clauses
      - Created 80 points (with clause intro injection)
      - Final valid chunks: 396

🗂️ Step 3: Preparing data for evaluation...
   ✅ Processed 396 law document ch

## 2. Chuẩn bị các query từ benchmark


In [69]:
# ===== MODELS TO EVALUATE =====
print("🤖 Setting up models for evaluation...")

models_to_evaluate = [
    {
        'name': 'minhquan6203/paraphrase-vietnamese-law',
        'type': 'transformers',  
        'description': 'Mô hình Sentence Similarity đã fine tune trên bộ luật pháp Việt Nam',
        'max_length': 512
    },
    {
        'name': 'sentence-transformers/paraphrase-multilingual-mpnet-base-v2',
        'type': 'sentence_transformers',
        'description': 'Mô hình cơ sở đa ngôn ngữ (base model)',
        'max_length': 512
    },
    {
        'name': 'huyhuy123/paraphrase-vietnamese-law-ALQAC',
        'type': 'transformers',
        'description': 'Fine-tuned trực tiếp trên mô hình paraphrase-vietnamese-law',
        'max_length': 512
    },
    {
        'name': 'namnguyenba2003/Vietnamese_Law_Embedding_finetuned_v3_256dims',
        'type': 'transformers',
        'description': 'Mô hình embedding luật Việt Nam với 256 dimensions',
        'max_length': 512
    },
    {
        'name': 'maiduchuy321/vietnamese-bi-encoder-fine-tuning-for-law-chatbot',
        'type': 'transformers',
        'description': 'Bi-encoder cho chatbot luật Việt Nam',
        'max_length': 512
    },
    {
        'name': 'BAAI/bge-m3',
        'type': 'sentence_transformers',
        'description': 'BGE-M3 - mô hình multilingual embedding hiện đại',
        'max_length': 8192
    }
]

print(f"✅ Prepared {len(models_to_evaluate)} models for evaluation:")
for i, model in enumerate(models_to_evaluate):
    print(f"   {i+1}. {model['name']}")
    print(f"      Type: {model['type']} | Max Length: {model['max_length']} tokens")
    print(f"      Description: {model['description']}")
    print()

print("🎯 All models support ≥512 tokens as required!")
print("💾 Embeddings will be stored in RAM (not vector database)")


🤖 Setting up models for evaluation...
✅ Prepared 6 models for evaluation:
   1. minhquan6203/paraphrase-vietnamese-law
      Type: transformers | Max Length: 512 tokens
      Description: Mô hình Sentence Similarity đã fine tune trên bộ luật pháp Việt Nam

   2. sentence-transformers/paraphrase-multilingual-mpnet-base-v2
      Type: sentence_transformers | Max Length: 512 tokens
      Description: Mô hình cơ sở đa ngôn ngữ (base model)

   3. huyhuy123/paraphrase-vietnamese-law-ALQAC
      Type: transformers | Max Length: 512 tokens
      Description: Fine-tuned trực tiếp trên mô hình paraphrase-vietnamese-law

   4. namnguyenba2003/Vietnamese_Law_Embedding_finetuned_v3_256dims
      Type: transformers | Max Length: 512 tokens
      Description: Mô hình embedding luật Việt Nam với 256 dimensions

   5. maiduchuy321/vietnamese-bi-encoder-fine-tuning-for-law-chatbot
      Type: transformers | Max Length: 512 tokens
      Description: Bi-encoder cho chatbot luật Việt Nam

   6. BAAI/bge-m

In [70]:
# Các query từ benchmark
benchmark_queries = [
    "Người bị bệnh tâm thần là người mất năng lực hành vi dân sự là đúng hay sai?",
    "Sự thỏa thuận của các bên không vi phạm điều cấm của pháp luật, không trái đạo đức xã hội thì được gọi là hợp đồng là đúng hay sai?",
    "Tiền phúng viếng đám ma cũng thuộc di sản thừa kế của người chết để lại là đúng hay sai?",
    "Cá nhân được hưởng di sản thừa kế phải trả nợ thay cho người để lại di sản là đúng hay sai?",
    "Hợp đồng vô hiệu là hợp đồng vi phạm pháp luật là đúng hay sai?",
    
    # Các query về Luật Hôn nhân & Gia đình (phù hợp với data LuatHonNhan)
    "Hoa lợi, lợi tức phát sinh từ tài sản riêng của một bên vợ hoặc chồng sẽ là tài sản chung nếu hoa lợi, lợi tức đó là nguồn sống duy nhất của gia đình.",
    "Hội Liên hiệp phụ nữ có quyền yêu cầu Tòa án ra quyết định hủy kết hôn trái pháp luật do vi phạm sự tự nguyện.",
    "Hôn nhân chỉ chấm dứt khi một bên vợ, chồng chết.",
    "Kết hôn có yếu tố nước ngoài có thể đăng ký tại UBND cấp xã.",
    "Khi cha mẹ không thể nuôi dưỡng, cấp dưỡng được cho con, thì ông bà phải có nghĩa vụ nuôi dưỡng hoặc cấp dưỡng cho cháu.",
    "Khi hôn nhân chấm dứt, mọi quyền và nghĩa vụ giữa những người đã từng là vợ chồng cũng chấm dứt.",
    "Khi vợ chồng ly hôn, con dưới 36 tháng tuổi được giao cho người vợ trực tiếp nuôi dưỡng.",
    "Khi vợ hoặc chồng thực hiện những giao dịch phục vụ cho nhu cầu thiết yếu của gia đình mà không có sự đồng ý của bên kia thì người thực hiện giao dịch đó phải thanh toán bằng tài sản riêng của mình.",
    "Khi không sống chung cùng với cha mẹ, con đã thành niên có khả năng lao động phải cấp dưỡng cho cha mẹ.",
    "Chỉ UBND cấp tỉnh nơi công dân Việt Nam cư trú mới có thẩm quyền đăng ký việc kết hôn giữa công dân Việt Nam với người nước ngoài.",
    
    # Thêm các query khác để test đa dạng hơn
    "Điều kiện kết hôn của nam và nữ theo pháp luật Việt Nam là gì?",
    "Tài sản nào được coi là tài sản chung của vợ chồng?",
    "Thủ tục ly hôn tại tòa án được quy định như thế nào?",
    "Quyền và nghĩa vụ của cha mẹ đối với con chưa thành niên?",
    "Trường hợp nào vợ chồng có thể thỏa thuận về chế độ tài sản?"
]

print(f"Prepared {len(benchmark_queries)} benchmark queries")
print("Sample queries:")
for i, query in enumerate(benchmark_queries[:5]):
    print(f"{i+1}. {query}")


Prepared 20 benchmark queries
Sample queries:
1. Người bị bệnh tâm thần là người mất năng lực hành vi dân sự là đúng hay sai?
2. Sự thỏa thuận của các bên không vi phạm điều cấm của pháp luật, không trái đạo đức xã hội thì được gọi là hợp đồng là đúng hay sai?
3. Tiền phúng viếng đám ma cũng thuộc di sản thừa kế của người chết để lại là đúng hay sai?
4. Cá nhân được hưởng di sản thừa kế phải trả nợ thay cho người để lại di sản là đúng hay sai?
5. Hợp đồng vô hiệu là hợp đồng vi phạm pháp luật là đúng hay sai?


## 3. Danh sách các mô hình cần đánh giá (≥512 tokens)


## 5. Search và Evaluation Functions


In [71]:
# ===== SEARCH AND EVALUATION FUNCTIONS =====
print("🔍 Setting up search and evaluation functions...")

def search_top_k(query_embedding, doc_embeddings, k=15):
    """
    Tìm top-k documents tương tự nhất với query (lưu trên RAM)
    
    Input:
    - query_embedding: vector embedding của query (1D array)
    - doc_embeddings: matrix embeddings của documents (2D array)
    - k: số lượng top results cần lấy
    
    Output:
    - top_indices: indices của top-k documents
    - top_scores: similarity scores của top-k documents
    """
    # Tính cosine similarity giữa query và tất cả documents
    similarities = cosine_similarity([query_embedding], doc_embeddings)[0]
    
    # Sắp xếp theo độ tương tự giảm dần và lấy top-k
    top_indices = np.argsort(similarities)[::-1][:k]
    top_scores = similarities[top_indices]
    
    return top_indices, top_scores

def calculate_metrics(scores, threshold_07=0.7, threshold_05=0.5):
    """
    Tính toán các metrics đánh giá chất lượng retrieval
    
    Input:
    - scores: array of similarity scores
    - threshold_07: ngưỡng cao (0.7)
    - threshold_05: ngưỡng thấp (0.5)
    
    Output:
    - dict chứa các metrics
    """
    return {
        "max_score": float(np.max(scores)),
        "avg_top3": float(np.mean(scores[:3])) if len(scores) >= 3 else float(np.mean(scores)),
        "avg_top5": float(np.mean(scores[:5])) if len(scores) >= 5 else float(np.mean(scores)),
        "avg_top10": float(np.mean(scores[:10])) if len(scores) >= 10 else float(np.mean(scores)),
        "avg_all": float(np.mean(scores)),
        "scores_above_07": int(np.sum(scores >= threshold_07)),
        "scores_above_05": int(np.sum(scores >= threshold_05)),
        "min_score": float(np.min(scores))
    }

def display_search_results(query, law_docs, top_indices, top_scores, max_display=5):
    """
    Hiển thị kết quả search một cách rõ ràng
    
    Input:
    - query: câu hỏi query
    - law_docs: list of law documents
    - top_indices: indices của top results
    - top_scores: similarity scores
    - max_display: số lượng results tối đa để hiển thị
    """
    print(f"📝 Query: {query}")
    print(f"🎯 Top {min(max_display, len(top_indices))} Results:")
    
    for i in range(min(max_display, len(top_indices))):
        idx = top_indices[i]
        score = top_scores[i]
        doc = law_docs[idx]
        
        print(f"\n   {i+1}. Score: {score:.4f} | {doc['title']}")
        print(f"      Type: {doc['type']} | Length: {doc['length']} chars")
        print(f"      Content: {doc['text'][:200]}...")
        
        # Hiển thị metadata nếu có
        if doc.get('metadata') and doc['metadata']:
            metadata_str = ", ".join([f"{k}: {v}" for k, v in doc['metadata'].items()])
            print(f"      Metadata: {metadata_str}")

print("✅ Search and evaluation functions ready!")


🔍 Setting up search and evaluation functions...
✅ Search and evaluation functions ready!


## 6. Model Evaluation Engine


In [ ]:
# ===== MODEL EVALUATION ENGINE =====
print("🧪 Setting up model evaluation engine...")

def evaluate_single_model(model_info, law_docs, queries, top_k=15, show_detailed_results=True):
    """
    Đánh giá một mô hình embedding trên dataset luật và benchmark queries
    
    Input:
    - model_info: dict chứa thông tin model (name, type, max_length...)
    - law_docs: list of law documents
    - queries: list of benchmark queries
    - top_k: số lượng top results để đánh giá
    - show_detailed_results: có hiển thị kết quả chi tiết không
    
    Output:
    - dict chứa results và metrics của model
    """
    print(f"\n{'='*80}")
    print(f"🔍 EVALUATING MODEL: {model_info['name']}")
    print(f"📝 Description: {model_info['description']}")
    print(f"🔧 Type: {model_info['type']} | Max Length: {model_info['max_length']} tokens")
    print(f"{'='*80}")
    
    try:
        # Bước 1: Chuẩn bị texts
        doc_texts = [doc['text'] for doc in law_docs]
        print(f"\n📚 Step 1: Prepared {len(doc_texts)} document texts")
        
        # Bước 2: Encode documents
        print(f"\n🔨 Step 2: Encoding documents...")
        if model_info['type'] == 'sentence_transformers':
            doc_embeddings = encode_with_sentence_transformers(
                doc_texts, 
                model_info['name'], 
                batch_size=16
            )
        else:
            doc_embeddings = encode_with_transformers(
                doc_texts, 
                model_info['name'], 
                max_length=model_info['max_length'],
                batch_size=16
            )
        
        print(f"   ✅ Document embeddings shape: {doc_embeddings.shape}")
        
        # Bước 3: Encode queries
        print(f"\n🔍 Step 3: Encoding queries...")
        if model_info['type'] == 'sentence_transformers':
            query_embeddings = encode_with_sentence_transformers(
                queries, 
                model_info['name'], 
                batch_size=16
            )
        else:
            query_embeddings = encode_with_transformers(
                queries, 
                model_info['name'], 
                max_length=model_info['max_length'],
                batch_size=16
            )
        
        print(f"   ✅ Query embeddings shape: {query_embeddings.shape}")
        
        # Bước 4: Evaluate từng query
        print(f"\n📊 Step 4: Evaluating {len(queries)} queries...")
        query_results = []
        all_metrics = []
        
        for i, query in enumerate(queries):
            print(f"\n   🔍 Query {i+1}/{len(queries)}")
            
            # Search top-k documents
            top_indices, top_scores = search_top_k(
                query_embeddings[i], 
                doc_embeddings, 
                k=top_k
            )
            
            # Calculate metrics
            metrics = calculate_metrics(top_scores)
            all_metrics.append(metrics)
            
            # Store results
            query_result = {
                'query': query,
                'query_id': i,
                'top_indices': top_indices.tolist(),
                'top_scores': top_scores.tolist(),
                'metrics': metrics
            }
            query_results.append(query_result)
            
            # Show detailed results for first few queries
            if show_detailed_results and i < 3:
                display_search_results(query, law_docs, top_indices, top_scores, max_display=3)
                print(f"      📈 Metrics: Max={metrics['max_score']:.4f}, Avg_top5={metrics['avg_top5']:.4f}, Above_0.7={metrics['scores_above_07']}")
        
        # Bước 5: Aggregate metrics
        print(f"\n📈 Step 5: Aggregating metrics...")
        
        # Calculate average metrics across all queries
        avg_metrics = {}
        metric_keys = all_metrics[0].keys()
        for key in metric_keys:
            if key.startswith('scores_above'):
                # For count metrics, sum then average
                avg_metrics[f"avg_{key}"] = np.mean([m[key] for m in all_metrics])
            else:
                # For score metrics, just average
                avg_metrics[f"avg_{key}"] = np.mean([m[key] for m in all_metrics])
        
        # Final result
        final_result = {
            'model_name': model_info['name'],
            'model_type': model_info['type'],
            'model_description': model_info['description'],
            'max_length': model_info['max_length'],
            'num_queries': len(queries),
            'num_documents': len(law_docs),
            'top_k': top_k,
            'query_results': query_results,
            'aggregated_metrics': avg_metrics,
            'evaluation_success': True
        }
        
        # Print summary
        print(f"\n✅ EVALUATION COMPLETED SUCCESSFULLY!")
        print(f"   📊 Average Results:")
        print(f"      - Avg Max Score: {avg_metrics['avg_max_score']:.4f}")
        print(f"      - Avg Top-5 Score: {avg_metrics['avg_avg_top5']:.4f}")
        print(f"      - Avg Above 0.7: {avg_metrics['avg_scores_above_07']:.1f}")
        print(f"      - Avg Above 0.5: {avg_metrics['avg_scores_above_05']:.1f}")
        
        return final_result
        
    except Exception as e:
        print(f"\n❌ EVALUATION FAILED: {str(e)}")
        return {
            'model_name': model_info['name'],
            'model_type': model_info['type'],
            'error': str(e),
            'evaluation_success': False
        }

print("✅ Model evaluation engine ready!")


🧪 Setting up model evaluation engine...
✅ Model evaluation engine ready!


## 7. Run Evaluation cho tất cả Models


In [73]:
# ===== RUN EVALUATION FOR ALL MODELS =====
print("🚀 Starting evaluation for all models...")

# Check available data
if not law_docs:
    print("❌ Error: No law documents loaded! Please run the data loading cells first.")
    evaluation_results = []
else:
    print(f"✅ Ready to evaluate with:")
    print(f"   📚 Documents: {len(law_docs)} law chunks")
    print(f"   ❓ Queries: {len(benchmark_queries)} benchmark questions")
    print(f"   🤖 Models: {len(models_to_evaluate)} models to test")
    print(f"   🎯 Top-K: 15 results per query")
    
    # Storage for results
    evaluation_results = []
    successful_evaluations = 0
    failed_evaluations = 0
    
    # Evaluate each model
    for i, model_info in enumerate(models_to_evaluate):
        print(f"\n{'🤖 '*20}")
        print(f"🤖 EVALUATING MODEL {i+1}/{len(models_to_evaluate)}")
        print(f"{'🤖 '*20}")
        
        try:
            # Run evaluation
            result = evaluate_single_model(
                model_info=model_info,
                law_docs=law_docs,
                queries=benchmark_queries,
                top_k=15,
                show_detailed_results=(i < 2)  # Show details for first 2 models only
            )
            
            if result['evaluation_success']:
                evaluation_results.append(result)
                successful_evaluations += 1
                print(f"✅ Model {i+1} evaluation completed successfully!")
            else:
                print(f"❌ Model {i+1} evaluation failed: {result.get('error', 'Unknown error')}")
                failed_evaluations += 1
            
            # Wait between models to prevent memory issues
            if i < len(models_to_evaluate) - 1:
                print(f"⏳ Waiting 2 seconds before next model...")
                time.sleep(2)
                
        except Exception as e:
            print(f"❌ Unexpected error evaluating model {i+1}: {str(e)}")
            failed_evaluations += 1
            continue
    
    # Final summary
    print(f"\n{'='*100}")
    print(f"🎉 EVALUATION SUMMARY")
    print(f"{'='*100}")
    print(f"✅ Successful evaluations: {successful_evaluations}")
    print(f"❌ Failed evaluations: {failed_evaluations}")
    print(f"📊 Total models evaluated: {len(evaluation_results)}")
    
    if evaluation_results:
        print(f"\n📈 Quick Performance Preview:")
        for result in evaluation_results:
            metrics = result['aggregated_metrics']
            model_name = result['model_name'].split('/')[-1]  # Get model name without path
            print(f"   🤖 {model_name}")
            print(f"      Max Score: {metrics['avg_max_score']:.4f} | Top-5: {metrics['avg_avg_top5']:.4f} | Above 0.7: {metrics['avg_scores_above_07']:.1f}")
    
    print(f"\n🎯 Ready for detailed analysis and report generation!")
    print(f"📊 Variable 'evaluation_results' contains all results for further analysis.")


🚀 Starting evaluation for all models...
✅ Ready to evaluate with:
   📚 Documents: 396 law chunks
   ❓ Queries: 20 benchmark questions
   🤖 Models: 6 models to test
   🎯 Top-K: 15 results per query

🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 
🤖 EVALUATING MODEL 1/6
🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 

🔍 EVALUATING MODEL: minhquan6203/paraphrase-vietnamese-law
📝 Description: Mô hình Sentence Similarity đã fine tune trên bộ luật pháp Việt Nam
🔧 Type: transformers | Max Length: 512 tokens

📚 Step 1: Prepared 396 document texts

🔨 Step 2: Encoding documents...
Loading model: minhquan6203/paraphrase-vietnamese-law


Encoding: 100%|██████████| 25/25 [00:02<00:00, 10.93it/s]


   ✅ Generated 396 embeddings, stored in RAM
   ✅ Document embeddings shape: (396, 768)

🔍 Step 3: Encoding queries...
Loading model: minhquan6203/paraphrase-vietnamese-law


Encoding: 100%|██████████| 2/2 [00:00<00:00, 32.77it/s]


   ✅ Generated 20 embeddings, stored in RAM
   ✅ Query embeddings shape: (20, 768)

📊 Step 4: Evaluating 20 queries...

   🔍 Query 1/20
📝 Query: Người bị bệnh tâm thần là người mất năng lực hành vi dân sự là đúng hay sai?
🎯 Top 3 Results:

   1. Score: 0.3415 | Điều 5, Khoản 2, Điểm h)
      Type: point | Length: 59 chars
      Content: Khoản 2, điểm h) Cấm các hành vi sau đây:
Bạo lực gia đình;...
      Metadata: chapter: Chương I, section: None, article_no: 5, article_title: Bảo vệ chế độ hôn nhân và gia đình, clause_no: 2, point_letter: h, exact_citation: Điều 5 khoản 2 điểm h), clause_intro: Cấm các hành vi sau đây:

   2. Score: 0.3082 | Điều 74 - Bồi thường thiệt hại do con gây ra
      Type: article_intro | Length: 187 chars
      Content: Điều 74. Bồi thường thiệt hại do con gây ra
Cha mẹ phải bồi thường thiệt hại do con chưa thành niên, con đã thành niên mất năng lực hành vi dân sự gây ra theo quy định của Bộ luật dân sự....
      Metadata: chapter: Chương V, section: Mục 1 – 

...

   ✅ Generated 20 embeddings, stored in RAM
   ✅ Query embeddings shape: (20, 768)

📊 Step 4: Evaluating 20 queries...

   🔍 Query 1/20

   🔍 Query 2/20

   🔍 Query 3/20

   🔍 Query 4/20

   🔍 Query 5/20

   🔍 Query 6/20

   🔍 Query 7/20

   🔍 Query 8/20

   🔍 Query 9/20

   🔍 Query 10/20

   🔍 Query 11/20

   🔍 Query 12/20

   🔍 Query 13/20

   🔍 Query 14/20

   🔍 Query 15/20

   🔍 Query 16/20

   🔍 Query 17/20

   🔍 Query 18/20

   🔍 Query 19/20

   🔍 Query 20/20

📈 Step 5: Aggregating metrics...

✅ EVALUATION COMPLETED SUCCESSFULLY!
   📊 Average Results:
      - Avg Max Score: 0.6379
      - Avg Top-5 Score: 0.5981
      - Avg Above 0.7: 2.2
      - Avg Above 0.5: 10.6
✅ Model 5 evaluation completed successfully!
⏳ Waiting 2 seconds before next model...

🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 
🤖 EVALUATING MODEL 6/6
🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 

🔍 EVALUATING MODEL: BAAI/bge-m3
📝 Description: BGE-M3 - mô hình multilingual embedding hiện đại
🔧 Type: sentence_transformers 

Batches:   0%|          | 0/25 [00:00<?, ?it/s]

   ✅ Generated 396 embeddings, stored in RAM
   ✅ Document embeddings shape: (396, 1024)

🔍 Step 3: Encoding queries...
Loading model: BAAI/bge-m3


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

   ✅ Generated 20 embeddings, stored in RAM
   ✅ Query embeddings shape: (20, 1024)

📊 Step 4: Evaluating 20 queries...

   🔍 Query 1/20

   🔍 Query 2/20

   🔍 Query 3/20

   🔍 Query 4/20

   🔍 Query 5/20

   🔍 Query 6/20

   🔍 Query 7/20

   🔍 Query 8/20

   🔍 Query 9/20

   🔍 Query 10/20

   🔍 Query 11/20

   🔍 Query 12/20

   🔍 Query 13/20

   🔍 Query 14/20

   🔍 Query 15/20

   🔍 Query 16/20

   🔍 Query 17/20

   🔍 Query 18/20

   🔍 Query 19/20

   🔍 Query 20/20

📈 Step 5: Aggregating metrics...

✅ EVALUATION COMPLETED SUCCESSFULLY!
   📊 Average Results:
      - Avg Max Score: 0.7181
      - Avg Top-5 Score: 0.6819
      - Avg Above 0.7: 4.7
      - Avg Above 0.5: 12.9
✅ Model 6 evaluation completed successfully!

🎉 EVALUATION SUMMARY
✅ Successful evaluations: 6
❌ Failed evaluations: 0
📊 Total models evaluated: 6

📈 Quick Performance Preview:
   🤖 paraphrase-vietnamese-law
      Max Score: 0.9034 | Top-5: 0.8780 | Above 0.7: 11.4
   🤖 paraphrase-multilingual-mpnet-base-v2
      Max

## 8. Analysis và Final Report


In [77]:
# ===== DETAILED ANALYSIS AND FINAL REPORT =====
print("📊 Generating detailed analysis and final report...")

if not evaluation_results:
    print("❌ No evaluation results available. Please run the evaluation first!")
else:
    print(f"\n{'='*100}")
    print(f"📋 COMPREHENSIVE EVALUATION REPORT")
    print(f"{'='*100}")
    
    # Sort models by average max score (best first)
    sorted_results = sorted(
        evaluation_results, 
        key=lambda x: x['aggregated_metrics']['avg_max_score'], 
        reverse=True
    )
    
    print(f"📊 DATASET INFORMATION:")
    print(f"   📚 Law Documents: {len(law_docs)} chunks from Luật Hôn nhân và Gia đình")
    print(f"   ❓ Benchmark Queries: {len(benchmark_queries)} questions")
    print(f"   🔍 Evaluation Method: Top-15 retrieval with cosine similarity")
    print(f"   💾 Storage: RAM-based (no vector database)")
    
    print(f"\n🏆 RANKING BY PERFORMANCE:")
    print(f"   Metric: Average Max Score across all queries")
    
    # Create comparison table
    print(f"\n{'Rank':<4} {'Model':<45} {'Max Score':<10} {'Top-5':<8} {'≥0.7':<6} {'≥0.5':<6} {'Type':<12}")
    print(f"{'-'*95}")
    
    for i, result in enumerate(sorted_results):
        metrics = result['aggregated_metrics']
        model_name = result['model_name'].split('/')[-1][:40]  # Truncate long names
        model_type = result['model_type']
        
        print(f"{i+1:<4} {model_name:<45} {metrics['avg_max_score']:<10.4f} "
              f"{metrics['avg_avg_top5']:<8.4f} {metrics['avg_scores_above_07']:<6.1f} "
              f"{metrics['avg_scores_above_05']:<6.1f} {model_type:<12}")
    
    # Best model analysis
    best_model = sorted_results[0]
    print(f"\n⭐ RECOMMENDED MODEL:")
    print(f"   🥇 {best_model['model_name']}")
    print(f"   📝 {best_model['model_description']}")
    print(f"   🎯 Performance Highlights:")
    best_metrics = best_model['aggregated_metrics']
    print(f"      - Average Max Score: {best_metrics['avg_max_score']:.4f}")
    print(f"      - Average Top-5 Score: {best_metrics['avg_avg_top5']:.4f}")
    print(f"      - Queries with score ≥ 0.7: {best_metrics['avg_scores_above_07']:.1f} per query")
    print(f"      - Queries with score ≥ 0.5: {best_metrics['avg_scores_above_05']:.1f} per query")
    
    # Performance analysis
    print(f"\n📈 PERFORMANCE ANALYSIS:")
    
    # Calculate overall statistics
    all_max_scores = [r['aggregated_metrics']['avg_max_score'] for r in evaluation_results]
    all_top5_scores = [r['aggregated_metrics']['avg_avg_top5'] for r in evaluation_results]
    
    print(f"   📊 Overall Statistics:")
    print(f"      - Best Max Score: {max(all_max_scores):.4f}")
    print(f"      - Worst Max Score: {min(all_max_scores):.4f}")
    print(f"      - Average Max Score: {np.mean(all_max_scores):.4f}")
    print(f"      - Best Top-5 Score: {max(all_top5_scores):.4f}")
    print(f"      - Average Top-5 Score: {np.mean(all_top5_scores):.4f}")
    
    # Model type analysis
    transformers_models = [r for r in evaluation_results if r['model_type'] == 'transformers']
    sentence_transformers_models = [r for r in evaluation_results if r['model_type'] == 'sentence_transformers']
    
    if transformers_models and sentence_transformers_models:
        print(f"\n🔧 MODEL TYPE COMPARISON:")
        
        trans_avg = np.mean([r['aggregated_metrics']['avg_max_score'] for r in transformers_models])
        sent_avg = np.mean([r['aggregated_metrics']['avg_max_score'] for r in sentence_transformers_models])
        
        print(f"   🔨 Transformers models: {len(transformers_models)} models, avg score: {trans_avg:.4f}")
        print(f"   📦 Sentence-transformers: {len(sentence_transformers_models)} models, avg score: {sent_avg:.4f}")
        
        if trans_avg > sent_avg:
            print(f"   ✅ Transformers models perform better on average (+{trans_avg - sent_avg:.4f})")
        else:
            print(f"   ✅ Sentence-transformers models perform better on average (+{sent_avg - trans_avg:.4f})")
    
    # Recommendations
    print(f"\n💡 RECOMMENDATIONS:")
    print(f"   🎯 For Production Deployment:")
    print(f"      - Primary: {best_model['model_name']}")
    print(f"      - Type: {best_model['model_type']}")
    print(f"      - Max Length: {best_model['max_length']} tokens")
    
    if len(sorted_results) > 1:
        second_best = sorted_results[1]
        print(f"   🥈 Alternative Option:")
        print(f"      - {second_best['model_name']}")
        print(f"      - Performance difference: {best_metrics['avg_max_score'] - second_best['aggregated_metrics']['avg_max_score']:.4f}")
    
    print(f"\n🔍 DETAILED QUERY ANALYSIS:")
    print(f"   📝 Sample Query Performance (Best Model):")
    
    # Show performance on first 3 queries for best model
    best_query_results = best_model['query_results'][:3]
    for i, qr in enumerate(best_query_results):
        print(f"\n   Query {i+1}: {qr['query'][:60]}...")
        print(f"      Max Score: {qr['metrics']['max_score']:.4f}")
        print(f"      Top-3 Average: {qr['metrics']['avg_top3']:.4f}")
        print(f"      Results ≥ 0.7: {qr['metrics']['scores_above_07']}")
    
    # Save summary to variables for further use
    evaluation_summary = {
        'best_model': best_model['model_name'],
        'best_score': best_metrics['avg_max_score'],
        'total_models_evaluated': len(evaluation_results),
        'total_queries': len(benchmark_queries),
        'total_documents': len(law_docs),
        'sorted_results': sorted_results
    }
    
    print(f"\n{'='*100}")
    print(f"✅ EVALUATION COMPLETED SUCCESSFULLY!")
    print(f"📊 Summary saved to 'evaluation_summary' variable")
    print(f"📋 Full results available in 'evaluation_results' variable")
    print(f"{'='*100}")
    
    # Export option
    print(f"\n💾 EXPORT OPTIONS:")
    print(f"   To save results to JSON file:")
    print(f"   → import json")
    print(f"   → with open('embedding_evaluation_results.json', 'w', encoding='utf-8') as f:")
    print(f"   →     json.dump(evaluation_results, f, ensure_ascii=False, indent=2)")
    
    print(f"\n🎉 Report generation completed!")
    
    # Make summary available globally
    globals()['evaluation_summary'] = evaluation_summary


📊 Generating detailed analysis and final report...

📋 COMPREHENSIVE EVALUATION REPORT
📊 DATASET INFORMATION:
   📚 Law Documents: 396 chunks from Luật Hôn nhân và Gia đình
   ❓ Benchmark Queries: 20 questions
   🔍 Evaluation Method: Top-15 retrieval with cosine similarity
   💾 Storage: RAM-based (no vector database)

🏆 RANKING BY PERFORMANCE:
   Metric: Average Max Score across all queries

Rank Model                                         Max Score  Top-5    ≥0.7   ≥0.5   Type        
-----------------------------------------------------------------------------------------------
1    paraphrase-vietnamese-law                     0.9034     0.8780   11.4   12.8   transformers
2    paraphrase-vietnamese-law-ALQAC               0.8711     0.7927   6.6    13.2   transformers
3    Vietnamese_Law_Embedding_finetuned_v3_25      0.8577     0.8287   12.5   15.0   transformers
4    paraphrase-multilingual-mpnet-base-v2         0.7647     0.7310   8.0    14.1   sentence_transformers
5    bge-m3 

## 9. Test với Single Query


In [35]:
# ===== TEST SINGLE QUERY =====
print("🧪 Quick test with single query...")

# Test query
test_query = "Điều kiện kết hôn của nam và nữ theo pháp luật Việt Nam là gì?"
print(f"📝 Test Query: {test_query}")

manual_model_name = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2" 

# Check evaluation summary
if 'evaluation_summary' in globals() and evaluation_summary:
    if manual_model_name:  
        best_model_name = manual_model_name
        print(f"🎯 Using manual model: {best_model_name}")
    else:
        best_model_name = evaluation_summary['best_model']
        print(f"🥇 Using best model: {best_model_name}")

    # Find model info
    best_model_info = None
    for model in models_to_evaluate:
        if model['name'] == best_model_name:
            best_model_info = model
            break

    if best_model_info:
        print(f"🔍 Testing single query with model `{best_model_info['name']}`...")

        # Encode query
        if best_model_info['type'] == 'sentence_transformers':
            test_query_embedding = encode_with_sentence_transformers([test_query], best_model_info['name'])[0]
        else:
            test_query_embedding = encode_with_transformers([test_query], best_model_info['name'], best_model_info['max_length'])[0]

        # Encode documents (re-encode cho demo, thực tế nên cache)
        doc_texts = [doc['text'] for doc in law_docs]
        if best_model_info['type'] == 'sentence_transformers':
            doc_embeddings = encode_with_sentence_transformers(doc_texts, best_model_info['name'])
        else:
            doc_embeddings = encode_with_transformers(doc_texts, best_model_info['name'], best_model_info['max_length'])

        # Search
        top_indices, top_scores = search_top_k(test_query_embedding, doc_embeddings, k=10)

        # Display results
        display_search_results(test_query, law_docs, top_indices, top_scores, max_display=5)

        # Metrics
        metrics = calculate_metrics(top_scores)
        print(f"\n📊 Metrics for this query:")
        print(f"   - Max Score: {metrics['max_score']:.4f}")
        print(f"   - Top-5 Average: {metrics['avg_top5']:.4f}")
        print(f"   - Results ≥ 0.7: {metrics['scores_above_07']}")
        print(f"   - Results ≥ 0.5: {metrics['scores_above_05']}")
    else:
        print("❌ Could not find model info for testing")

else:
    print("⚠️ No evaluation results available. Run the full evaluation first!")

print(f"\n✅ Single query test completed!")


🧪 Quick test with single query...
📝 Test Query: Điều kiện kết hôn của nam và nữ theo pháp luật Việt Nam là gì?
🎯 Using manual model: sentence-transformers/paraphrase-multilingual-mpnet-base-v2
🔍 Testing single query with model `sentence-transformers/paraphrase-multilingual-mpnet-base-v2`...
Loading model: sentence-transformers/paraphrase-multilingual-mpnet-base-v2


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

   ✅ Generated 1 embeddings, stored in RAM
Loading model: sentence-transformers/paraphrase-multilingual-mpnet-base-v2


Batches:   0%|          | 0/13 [00:00<?, ?it/s]

   ✅ Generated 396 embeddings, stored in RAM
📝 Query: Điều kiện kết hôn của nam và nữ theo pháp luật Việt Nam là gì?
🎯 Top 5 Results:

   1. Score: 0.8354 | Điều 126, Khoản 1
      Type: clause | Length: 309 chars
      Content: Khoản 1. Trong việc kết hôn giữa công dân Việt Nam với người nước ngoài, mỗi bên phải tuân theo pháp luật của nước mình về điều kiện kết hôn; nếu việc kết hôn được tiến hành tại cơ quan nhà nước có th...
      Metadata: chapter: Chương VIII, section: None, article_no: 126, article_title: Kết hôn có yếu tố nước ngoài, clause_no: 1, exact_citation: Điều 126 khoản 1

   2. Score: 0.8299 | Điều 126, Khoản 2
      Type: clause | Length: 173 chars
      Content: Khoản 2. Việc kết hôn giữa những người nước ngoài thường trú ở Việt Nam tại cơ quan có thẩm quyền của Việt Nam phải tuân theo các quy định của Luật này về điều kiện kết hôn....
      Metadata: chapter: Chương VIII, section: None, article_no: 126, article_title: Kết hôn có yếu tố nước ngoài, clause_no: 2, exac